In [ ]:
%matplotlib inline
import sys
import os

import pandas as pd
import numpy as np
import json
from datetime import datetime

import matplotlib.pyplot as plt

COLORS = [(0.12109375, 0.46484375, 0.703125),
          (0.99609375, 0.49609375, 0.0546875),
          (0.171875, 0.625, 0.171875),
          (0.8359375, 0.15234375, 0.15625),
          (0.578125, 0.40234375, 0.73828125),
          (0.546875, 0.3359375, 0.29296875),
          (0.88671875, 0.46484375, 0.7578125),
          (0.49609375, 0.49609375, 0.49609375),
          (0.734375, 0.73828125, 0.1328125),
          (0.08984375, 0.7421875, 0.80859375)]

import calendar

# Set font sizes
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20
                
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
CARBON_INTENSITY = {"biogas":18, "biomass":18, "geo":42, "hydro":4,
                        "imports":428, "nuclear":16, "smhydro":4, "solarpv":46, "solarth":22,
                        "thermal":469, "wind":12}

df_carbon = pd.read_csv("../data/carbon_scenarios/carbon_x2.csv", index_col=0, parse_dates=True)
df_carbon.index -= pd.Timedelta('7h')

df_carbon.dropna(inplace=True)
df_carbon.loc[:, 'year'] = df_carbon.index.year
df_carbon.loc[:, 'month'] = df_carbon.index.month
df_carbon.loc[:, 'hour'] = df_carbon.index.hour

print(df_carbon.head())

In [ ]:
def loadCAISO(year):
    dfp = pd.read_csv(os.path.join("../data/CAISO_DailyRenewablesWatch", 'DailyRenewablesWatch_%d.csv' % year),
                      index_col=0, parse_dates=True)
    dfp.index -= pd.Timedelta('7h')

    cols = [col for col in dfp.columns if col != 'carbon']
    dfp["total"] = dfp[cols].sum(axis=1)
    CARBON_INTENSITY = {"biogas":18, "biomass":18, "geo":42, "hydro":4,
                        "imports":428, "nuclear":16, "smhydro":4, "solarpv":46, "solarth":22,
                        "thermal":469, "wind":12}

    # recalculate carbon without the exports
    dfp["carbon"] = dfp.apply(lambda row:sum(row[fuel]*CARBON_INTENSITY[fuel]
                                           for fuel in CARBON_INTENSITY)/1e3, axis=1)
    dfp["carbon_intensity"] = dfp.apply(lambda row:row["carbon"]*1e3/row["total"], axis=1)
    return dfp

df = pd.concat([loadCAISO(y) for y in [2015,2016,2017,2018]])

df.dropna(inplace=True)
df.loc[:, 'year'] = df.index.year
df.loc[:, 'month'] = df.index.month
df.loc[:, 'hour'] = df.index.hour

In [ ]:
# generate MEFs for 2016 and 2025
df_carbon["MEF"] = (CARBON_INTENSITY["solarpv"]* ((df_carbon.index.hour < 17) & (df_carbon.index.hour > 8))
                    + CARBON_INTENSITY["thermal"] * ((df_carbon.index.hour > 16) | (df_carbon.index.hour < 9)))
df["MEF"] = CARBON_INTENSITY["thermal"]

In [ ]:
grp_15 = df.loc[pd.to_datetime("2015-01-01"):pd.to_datetime("2015-12-01"),
                ["carbon_intensity", "year", "hour"]].groupby(["year", "hour"]).mean()
grp_16 = df.loc[pd.to_datetime("2016-01-01"):pd.to_datetime("2016-12-01"),
                ["carbon_intensity", "year", "hour"]].groupby(["year", "hour"]).mean()
grp_17 = df.loc[pd.to_datetime("2017-01-01"):pd.to_datetime("2017-12-01"),
                ["carbon_intensity", "year", "hour"]].groupby(["year", "hour"]).mean()
grp_18 = df.loc[pd.to_datetime("2018-01-01"):pd.to_datetime("2018-12-01"),
                ["carbon_intensity", "year", "hour"]].groupby(["year", "hour"]).mean()
grp_25 = df_carbon.loc[:,["carbon_intensity", "year", "hour"]].groupby(["year", "hour"]).mean()

MEF_16 = df.loc[pd.to_datetime("2016-01-01"):pd.to_datetime("2016-12-01"),
                ["MEF", "year", "hour"]].groupby(["year", "hour"]).mean()
MEF_25 = df_carbon.loc[:,["MEF", "year", "hour"]].groupby(["year", "hour"]).mean()

# April of each year, all on the same plot
#f, ax = plt.subplots(1, 1, figsize=(6,4))
f, ax = plt.subplots(1, 1)
ax.axvspan(0, 6, facecolor='b', alpha=0.05)
ax.axvspan(19, 23, facecolor='b', alpha=0.05)
ax.axvspan(6, 19, facecolor='y', alpha=0.05)
ax.text(10,310, "DAYTIME")

arrowprops=dict(color=(.33,.33,.33), shrink=0.05, width=1, headwidth=10)
ax.annotate('pre-2025\nMEFs', xy=(10, CARBON_INTENSITY["thermal"]), xytext=(1.5,340),
            arrowprops=arrowprops,
            fontsize=14)
ax.annotate('post-2025\nMEFs', xy=(8.5, 200), xytext=(.5,90),
            arrowprops=arrowprops,
            fontsize=14)

for igrp, grp in zip([2015,2016, 2017, 2018, 2025],
                     [grp_15,grp_16, grp_17, grp_18, grp_25]):
    ls="-"
    sel = igrp
    if igrp == 2025:
        ls = "--"
        sel = 2016
    ax.plot(grp.loc[sel, "carbon_intensity"], label=igrp, lw=2, ls=ls)

ax.plot(MEF_16.loc[2016, "MEF"], label="__nolegend__", color=(.33,.33,.33), ls = "-")
ax.plot(MEF_25.loc[2016, "MEF"], label="__nolegend__", marker="o", color=(.33,.33,.33), ls = "--")
ax.grid(True)
ax.legend(loc=2, bbox_to_anchor=(1.0, 0.8))
ax.set_xlim([0,23])
ax.set_ylim([0,500])
ax.set_xlabel('Hour of the day')
ax.set_title('California: hourly AEFs and MEFs');
ax.set_ylabel('kg/MWh');
ax.set_yticks([0, 200, 400])
ax.set_yticks([CARBON_INTENSITY["solarpv"], CARBON_INTENSITY["thermal"]], minor=True)
ax.set_yticklabels(["Solar", "Gas"], minor=True)

#plt.tight_layout()
plt.savefig('figures/fig1.pdf', bbox_inches='tight')
plt.savefig('figures/fig1.png', bbox_inches='tight')

In [ ]:
# Split into different years
df16 = df.loc[pd.to_datetime("2016-01-01"):pd.to_datetime("2017-01-01"),:].copy(deep=True)
df18 = df.loc[pd.to_datetime("2018-01-01"):pd.to_datetime("2019-12-01"),:].copy(deep=True)
df25 = df.loc[pd.to_datetime("2016-01-01"):pd.to_datetime("2017-01-01"),:].copy(deep=True)
df25["carbon_intensity"] = df_carbon["carbon_intensity"]
df25["MEF"] = df_carbon["MEF"]

In [ ]:
# Hourly Emissions analysis (in ktonnes)

# Calculate references (2016)
footprint_h_16 = df16.carbon_intensity.sum() * 1e-6
footprint_y_16 = df16.carbon_intensity.mean() * len(df16.solarpv) * 1e-6

# Choose scenario
def calcs(df, verb=0):
    # Scale wind and solar data to get generation
    df.loc[:,"wind_100"] = df.loc[:, "wind"] * len(df) / df.wind.sum()
    df.loc[:,"wind_50"] = 0.5 * df.loc[:, "wind"] * len(df) / df.wind.sum()
    df.loc[:,"solarpv_100"] = df.loc[:, "solarpv"] * len(df) / df.solarpv.sum()
    df.loc[:,"solarpv_50"] = 0.5 * df.loc[:, "solarpv"] * len(df) / df.solarpv.sum()
    
    # Hourly calcs
    footprint_h = 1 * df.carbon_intensity.sum() * 1e-6
    df.loc[:,"avoided100_s_h"] = df.solarpv_100 * (df.MEF - CARBON_INTENSITY['solarpv'])
    df.loc[:,"avoided100_w_h"] = df.wind_100 * (df.MEF - CARBON_INTENSITY['wind'])
    df.loc[:,"avoided5050_h"] = (df.solarpv_50 * (df.MEF - CARBON_INTENSITY['solarpv'])
                                 + df.wind_50 * (df.MEF - CARBON_INTENSITY['wind']))
    avoided100_s_h = np.nansum(df.avoided100_s_h) * 1e-6
    avoided100_w_h = np.nansum(df.avoided100_w_h) * 1e-6
    avoided5050_h = np.nansum(df.avoided5050_h) * 1e-6
    
    # Note: I multiply by 1MW because I am considering a 1MW constant load in this study
    df.loc[:,"footprint100_s_h"] = 1 * df.carbon_intensity - df.avoided100_s_h
    df.loc[:,"footprint100_w_h"] = 1 * df.carbon_intensity - df.avoided100_w_h
    df.loc[:,"footprint5050_h"] = 1 * df.carbon_intensity - df.avoided5050_h
    
    footprint_100_s_h = footprint_h - avoided100_s_h
    footprint_100_w_h = footprint_h - avoided100_w_h
    footprint_5050_h = footprint_h - avoided5050_h
    
    if verb > 0:
        print("Hourly")
        print("Emissions footprint: %g" % footprint_h)
        print("Avoided tons 100 %% solar: %g" % avoided100_s_h)
        print("Avoided tons 100 %% wind: %g" % avoided100_w_h)
        print("Avoided tons 50 %% wind, 50 %% solar: %g" % avoided5050_h)

    # Yearly calcs
    GRID_AVG_CARBON = df.carbon_intensity.mean()
    GRID_AVG_MEF = df.MEF.mean()
    footprint_y = GRID_AVG_CARBON * len(df.carbon_intensity) * 1e-6
    df.loc[:,"avoided100_s_y"] = df.solarpv_100 * (GRID_AVG_MEF-CARBON_INTENSITY['solarpv'])
    df.loc[:,"avoided100_w_y"] = df.wind_100 * (GRID_AVG_MEF-CARBON_INTENSITY['wind'])
    df.loc[:,"avoided5050_y"] = (df.solarpv_50 * (GRID_AVG_MEF-CARBON_INTENSITY['solarpv'])
                                 + df.wind_50 * (GRID_AVG_MEF-CARBON_INTENSITY['wind']))
    avoided100_s_y =  np.nansum(df.avoided100_s_y) * 1e-6
    avoided100_w_y = np.nansum(df.avoided100_w_y) * 1e-6
    avoided5050_y = np.nansum(df.avoided5050_y) * 1e-6
    footprint_100_s_y = footprint_y-avoided100_s_y
    footprint_100_w_y = footprint_y-avoided100_w_y
    footprint_5050_y = footprint_y-avoided5050_y
    
    if verb > 0:
        print("\nYearly")
        print("Emissions footprint: %g" % footprint_y)
        print("Avoided tons 100 %% solar: %g" % avoided100_s_y)
        print("Avoided tons 100 %% wind: %g" % avoided100_w_y)
        print("Avoided tons 50 %% wind, 50 %% solar: %g" % avoided5050_y)

    # Summary dataframe to hold the results
    df_sum = pd.DataFrame(
        index=["Grid", "solar100", "wind100", "sw5050"],
        columns=["net_footprint_H", "credit_H", "red_H",
                 "net_footprint_Y", "credit_Y", "red_Y"])

    df_sum.loc["Grid",:] = [footprint_h, 0., (footprint_h_16-footprint_h)/footprint_h_16,
                            footprint_y, 0.,(footprint_y_16-footprint_y)/footprint_y_16]
    df_sum.loc["solar100",:] = [footprint_100_s_h, avoided100_s_h, (footprint_h_16-footprint_100_s_h)/footprint_h_16,
                            footprint_100_s_y, avoided100_s_y,(footprint_y_16-footprint_100_s_y)/footprint_y_16]
    df_sum.loc["wind100",:] = [footprint_100_w_h, avoided100_w_h, (footprint_h_16-footprint_100_w_h)/footprint_h_16,
                            footprint_100_w_y, avoided100_w_y, (footprint_y_16-footprint_100_w_y)/footprint_y_16]
    df_sum.loc["sw5050",:] = [footprint_5050_h, avoided5050_h, (footprint_h_16-footprint_5050_h)/footprint_h_16,
                            footprint_5050_y, avoided5050_y, (footprint_y_16-footprint_5050_y)/footprint_y_16]
    return df_sum

df_sum16 = calcs(df16)
df_sum18 = calcs(df18)
df_sum25 = calcs(df25)

In [ ]:
df_sum16.to_csv('figures/df_sum16.csv')
df_sum16

In [ ]:
df_sum18.to_csv('figures/df_sum18.csv')
df_sum18

In [ ]:
df_sum25.to_csv('figures/df_sum25.csv')
df_sum25

In [ ]:
# Sanity: the credit in 2016 and 2018 for 100% solar should be easy to compute, since the MEF is constant
1*(469-46)*8760 * 1e-6

In [ ]:
df18.month = df18.index.month
df18.hour = df18.index.hour
grped = df18.loc[:,[
        "carbon_intensity","avoided100_s_h", "avoided100_w_h",
        "avoided5050_h", "footprint100_s_h", "footprint100_w_h",
        "footprint5050_h", "month", "hour"]].groupby([
        "month", "hour"]).mean()
f, axes = plt.subplots(2, 3, figsize=(15,6))
for ax in axes.flatten():
    ax.axvspan(0, 6, facecolor='b', alpha=0.05)
    ax.axvspan(19, 23, facecolor='b', alpha=0.05)
    ax.axvspan(6, 19, facecolor='y', alpha=0.05)
    
for m, ax in zip([1, 5, 8], axes[0]):
    ax.plot([0,23],[0,0], lw=2, label="No gen.", color=COLORS[0])
    ax.plot(grped.loc[m, "avoided100_w_h"], lw=2, label="100% wind", color=COLORS[2])
    ax.plot(grped.loc[m, "avoided5050_h"], lw=2, label="50-50%", color=COLORS[1])
    ax.plot(grped.loc[m, "avoided100_s_h"], lw=2, label="100% solar", color=COLORS[3])
    
    ax.text(7,500, "DAYTIME - %s" % calendar.month_abbr[m].upper())

for m, ax in zip([1, 5, 8], axes[1]):
    ax.plot(grped.loc[m, "carbon_intensity"], lw=2, label="No gen.", color=COLORS[0])
    ax.plot(grped.loc[m, "footprint100_w_h"], lw=2, label="100% wind", color=COLORS[2])
    ax.plot(grped.loc[m, "footprint5050_h"], lw=2, label="50-50%", color=COLORS[1])
    ax.plot(grped.loc[m, "footprint100_s_h"], lw=2, label="100% solar", color=COLORS[3])
    
    
    ax.text(7,250, "DAYTIME - %s" % calendar.month_abbr[m].upper())
    
for ax in axes.flatten():
    ax.set_xlim([0,23])
    ax.grid(True)
    
for ax in axes[0]:
    ax.set_ylim([-8,1400])
    ax.set_ylabel('CO2 credit (kg)')

for ax in axes[1]:
    ax.set_ylim([-1200,500])
    ax.set_ylabel('CO2 footprint (kg)')
    ax.set_xlabel('hour')

axes[-1][0].legend(loc=3)#, bbox_to_anchor=(1.7, 0.5))

plt.tight_layout()
plt.savefig('figures/fig2a.pdf')
plt.savefig('figures/fig2a.png')

In [ ]:
df25.month = df25.index.month
df25.hour = df25.index.hour

grped = df25.loc[:,[
        "carbon_intensity","avoided100_s_h", "avoided100_w_h",
        "avoided5050_h", "footprint100_s_h", "footprint100_w_h",
        "footprint5050_h", "month", "hour"]].groupby([
        "month", "hour"]).mean()
f, axes = plt.subplots(2, 3, figsize=(15,6))
for ax in axes.flatten():
    ax.axvspan(0, 6, facecolor='b', alpha=0.05)
    ax.axvspan(19, 23, facecolor='b', alpha=0.05)
    ax.axvspan(6, 19, facecolor='y', alpha=0.05)
    
for m, ax in zip([1, 5, 8], axes[0]):
    ax.plot([0,23],[0,0], lw=2, label="No gen.", color=COLORS[0])
    ax.plot(grped.loc[m, "avoided100_w_h"], lw=2, label="100% wind", color=COLORS[2])
    ax.plot(grped.loc[m, "avoided5050_h"], lw=2, label="50-50%", color=COLORS[1])
    ax.plot(grped.loc[m, "avoided100_s_h"], lw=2, label="100% solar", color=COLORS[3])
    
    ax.text(7,500, "DAYTIME - %s" % calendar.month_abbr[m].upper())

for m, ax in zip([1, 5, 8], axes[1]):
    ax.plot(grped.loc[m, "carbon_intensity"], lw=2, label="No gen.", color=COLORS[0])
    ax.plot(grped.loc[m, "footprint100_w_h"], lw=2, label="100% wind", color=COLORS[2])
    ax.plot(grped.loc[m, "footprint5050_h"], lw=2, label="50-50%", color=COLORS[1])
    ax.plot(grped.loc[m, "footprint100_s_h"], lw=2, label="100% solar", color=COLORS[3])
    ax.text(7,250, "DAYTIME - %s" % calendar.month_abbr[m].upper())
    
for ax in axes.flatten():
    ax.set_xlim([0,23])
    ax.grid(True)
    
for ax in axes[0]:
    ax.set_ylim([-8,1400])
    ax.set_ylabel('CO2 credit (kg)')

for ax in axes[1]:
    ax.set_ylim([-1200,500])
    ax.set_ylabel('CO2 footprint (kg)')
    ax.set_xlabel('hour')

axes[-1][0].legend(loc=3)#, bbox_to_anchor=(1.7, 0.5))

plt.tight_layout()
plt.savefig('figures/fig2b.pdf')
plt.savefig('figures/fig2b.png')
